# **음원 분리 모델(최근) + stt(파인튜닝 적용)**

In [ ]:
# =========================
# Soribom AI Server – PyTorch Whisper Large-v3 + DCCRNet
# =========================

import os
import time
import re
import threading
from threading import Thread

# -------------------------
# 1. 초기화
# -------------------------
!fuser -k 5000/tcp 2>/dev/null
!pkill -f cloudflared 2>/dev/null
!pkill -f flask 2>/dev/null
if os.path.exists("tunnel.log"): os.remove("tunnel.log")
print("🔄 초기화 완료")

# -------------------------
# 2. 라이브러리 설치
# -------------------------
!pip install torch==2.3.1+cu121 torchaudio==2.3.1 --extra-index-url https://download.pytorch.org/whl/cu121 -q
!apt-get install -y ffmpeg > /dev/null
!pip install asteroid flask flask-cors nest_asyncio requests numpy soundfile librosa transformers -q

!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb > /dev/null
print("📦 설치 완료")

# -------------------------
# 3. Imports
# -------------------------
import torch, torchaudio, numpy as np
from asteroid.models import BaseModel
from flask import Flask, request, jsonify
from flask_cors import CORS
import nest_asyncio
from google.colab import drive
import librosa, soundfile as sf
from transformers import (
    WhisperForConditionalGeneration,
    WhisperFeatureExtractor,
    WhisperTokenizer,
    WhisperProcessor
)

nest_asyncio.apply()

# -------------------------
# 4. Config
# -------------------------
SAMPLE_RATE = int(os.getenv("SAMPLE_RATE", 16000))
HPF_CUTOFF = int(os.getenv("HPF_CUTOFF", 100))
DEVICE = os.getenv("DEVICE", "cuda")
MODE = os.getenv("MODE", "prod")

BACKEND_URL = os.getenv("BACKEND_URL", "http://127.0.0.1:8000")
CHECKPOINT = os.getenv("DCCRN_CHECKPOINT_PATH")
WHISPER_MODEL_PATH = os.getenv("WHISPER_MODEL_PATH")

print(f"⚙️ Device: {DEVICE}")
print("✔ BACKEND_URL =", BACKEND_URL)

# -------------------------
# 5. Mount Drive
# -------------------------
#if not os.path.exists('/content/drive'):
#    drive.mount('/content/drive')

# -------------------------
# 6. Load Enhancement 모델 (DCCRNet)
# -------------------------
def load_dccrn():
    print(f"📥 Enhancement 모델 로딩 중... 경로: {CHECKPOINT}")
    model = BaseModel.from_pretrained("JorisCos/DCCRNet_Libri1Mix_enhsingle_16k")
    ckpt = torch.load(CHECKPOINT, map_location=DEVICE)
    state_dict = ckpt.get('model_state_dict', ckpt)
    model.load_state_dict(state_dict, strict=True)
    model.to(DEVICE).eval()
    print("✅ Enhancement 모델 로드 완료!")
    return model

try:
    enhance_model = load_dccrn()
except Exception as e:
    print(f"❌ DCCRNet 모델 로드 실패: {e}")
    enhance_model = None
    print("⚠️ Enhancement 기능 비활성화됨.")

# -------------------------
# 7. Load Whisper large-v3 Fine-tuned 모델 (영어+한국어 다국어)
# -------------------------
print("📥 Whisper large-v3 모델 로딩 중...")

# Feature extractor + tokenizer 모두 원본 large-v3 사용 → 다국어 모드
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-large-v3")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-large-v3")
whisper_processor = WhisperProcessor(feature_extractor=feature_extractor, tokenizer=tokenizer)

# Fine-tuned 모델 가중치만 적용
whisper_model = WhisperForConditionalGeneration.from_pretrained(WHISPER_MODEL_PATH).to(DEVICE)

# forced_decoder_ids 제거 → 언어 자동 감지 모드
forced_decoder_ids = None

print("✅ Whisper large-v3 로드 완료! (영어+한국어 STT)")

# -------------------------
# 8. Flask Router
# -------------------------
app = Flask(__name__)
CORS(app)

@app.route("/ping")
def ping():
    return {"status": "alive"}, 200

@app.route("/enhance_stt", methods=["POST"])
def enhance_stt():
    try:
        if 'audio' not in request.files:
            return jsonify({"error": "audio file missing"}), 400

        audio_file = request.files["audio"]
        chunk_id = request.form.get("chunk_id", "unknown")
        print(f"🎧 Received chunk_id={chunk_id}")

        temp_path = f"temp_{chunk_id}.wav"
        audio_file.save(temp_path)

        # Load audio
        try:
            wav_np, sr = librosa.load(temp_path, sr=SAMPLE_RATE, mono=True)
        except:
            data, sr2 = sf.read(temp_path)
            wav_np = librosa.resample(data.T, orig_sr=sr2, target_sr=SAMPLE_RATE)
            if wav_np.ndim > 1: wav_np = np.mean(wav_np, axis=0)

        wav = torch.from_numpy(wav_np).float().unsqueeze(0).to(DEVICE)

        # HPF
        wav = torchaudio.functional.highpass_biquad(wav, SAMPLE_RATE, cutoff_freq=HPF_CUTOFF)

        # Normalization
        rms = torch.sqrt(torch.mean(wav**2) + 1e-8)
        wav = wav * (1 / (rms + 1e-8))

        # Enhancement
        if enhance_model:
            print("💡 Enhancement 실행 중...")
            with torch.no_grad():
                enh = enhance_model(wav.unsqueeze(0))
            if isinstance(enh, dict): enh = enh.get("waveform", list(enh.values())[0])
            if isinstance(enh, (list, tuple)): enh = enh[0]
            enh_np = enh.squeeze().cpu().numpy().astype(np.float32)
        else:
            enh_np = wav.squeeze().cpu().numpy().astype(np.float32)

        # Peak norm
        max_val = np.max(np.abs(enh_np))
        if max_val > 0:
            enh_np = enh_np / max_val * 0.9

        # -------------------------
        # Clean 파일 저장
        # -------------------------
        clean_path = f"clean_{chunk_id}.wav"
        torchaudio.save(clean_path, torch.tensor(enh_np).unsqueeze(0), SAMPLE_RATE)
        print(f"💾 Clean 파일 저장 완료 → {clean_path}")

        # -------------------------
        # Django로 clean 업로드
        # -------------------------
        try:
            up = requests.post(
                f"{BACKEND_URL}/api/save-clean-audio/",
                files={"file": open(clean_path, "rb")},
                data={"chunk_id": chunk_id},
                timeout=10
            )
            print("⬆ Clean 업로드:", up.status_code, up.text)
        except Exception as e:
            print("❌ Clean 업로드 실패:", e)

        # -------------------------
        # Whisper STT (영어+한국어 다국어)
        # -------------------------
        inputs = whisper_processor(enh_np, sampling_rate=SAMPLE_RATE, return_tensors="pt", task="transcribe").to(DEVICE)
        with torch.no_grad():
            predicted_ids = whisper_model.generate(**inputs, forced_decoder_ids=forced_decoder_ids)
        text = whisper_processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

        if os.path.exists(temp_path): os.remove(temp_path)
        print(f"📝 [Chunk {chunk_id}] {text}")
        return jsonify({"text": text, "confidence": 1.0})

    except Exception as e:
        import traceback
        traceback.print_exc()
        if 'temp_path' in locals() and os.path.exists(temp_path): os.remove(temp_path)
        return jsonify({"error": str(e)}), 500

# -------------------------
# 9. Run Flask
# -------------------------
print("🟢 Flask 라우팅 완료")
Thread(target=lambda: app.run(host="0.0.0.0", port=5000, debug=False, use_reloader=False),
       daemon=True).start()
time.sleep(4)

# -------------------------
# 10. Cloudflare Tunnel
# -------------------------
print("🌍 Cloudflare 실행 중...")
os.system("cloudflared tunnel --url http://localhost:5000 --logfile tunnel.log &")
time.sleep(7)
log = open("tunnel.log").read()
urls = re.findall(r"https://[0-9a-zA-Z\-]+\.trycloudflare\.com", log)
if urls:
    print("🚀 AI SERVER URL =", urls[-1])
else:
    print("❌ URL 생성 실패")
